In [230]:
import tensorflow.keras as keras
import numpy as np
from sklearn.utils import resample
import pandas as pd
import pickle
import os
import csv
import scipy.stats as stats
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import confusion_matrix,accuracy_score
from sklearn.linear_model import LogisticRegression
from statistics import mean
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Dropout
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import SGD
from keras.regularizers import l2
MODEL_PATH = './model/'
DATA_PATH = './data/'

In [231]:
pur_data = np.load(DATA_PATH+'purchase100.npz')
features = pur_data['features']
labels = pur_data['labels']
data = pd.DataFrame(features[:,:])
labels = np.argmax(labels, axis=1)
data['600'] = labels

In [232]:
target_dataset = pd.DataFrame(data.iloc[1:20001,].values)
shadow_dataset = pd.DataFrame(data.iloc[15001:50001,].values)
attack_test_nonmembers = pd.DataFrame(data.iloc[75001:85001,].values)
attack_test_members = pd.DataFrame(data.iloc[5001:15001,].values)

In [ ]:
data = pd.read_csv('data/adult.data', na_values=["?"])
target_dataset = pd.DataFrame(data.iloc[1:10001,].values)
shadow_dataset = pd.DataFrame(data.iloc[15001:30001,].values)
attack_test_nonmembers = pd.DataFrame(data.iloc[11001:14001,].values)
attack_test_members = pd.DataFrame(data.iloc[2001:5001,].values)

In [ ]:
target_dataset = pd.read_csv('data/adult_sds.csv', na_values=["?"])
shadow_dataset = pd.read_csv('data/adultODS10K_to_25K.csv', na_values=["?"])
attack_test_nonmembers = pd.read_csv('data/adultODS25K_to_28K.csv', na_values=["?"])
attack_test_members = pd.read_csv('data/adultODS1_to_3K.csv', na_values=["?"])

In [233]:
def read_data(data_name):
    with np.load(DATA_PATH + data_name) as f:
        train_x, train_y, test_x, test_y = [f['arr_%d' % i] for i in range(len(f.files))]
    return train_x, train_y, test_x, test_y

In [234]:
def transform_data(dataset):
    x = dataset.iloc[:,0:dataset.shape[1]-1] # seperate the feature column from class label
    y = dataset.iloc[:,-1] # label column

    dim=x.shape[1] # number of feature columns

    #num of classes
    num_classes=2

    for j in range(0,dim):
        if x.iloc[:,j].dtypes == object:   # transform categorical variables
            x.iloc[:,j] = x.iloc[:,j].astype('category') # change datatype to categorical
            x.iloc[:,j] = x.iloc[:,j].cat.codes # change from category name to category number
        else:  #transform numrical variables to standard scaler form 
            sc = StandardScaler()  
            val=np.array(x.iloc[:,j]).reshape(-1,1)
            std_data = sc.fit_transform(val)
            std_data = pd.DataFrame(std_data)
            x.iloc[:,j]=std_data

    y = y.astype('category') # change label to categorical
    y = y.cat.codes # change from category name to number

    x=np.array(x)
    y=np.array(y)
    
    
    y=to_categorical(y)
    
    return x, y, dim

In [235]:
def transform_puchase_data(dataset): 
    df_tot = dataset
    df_tot.dropna(inplace=True)

    trainX = df_tot.iloc[:,0:dataset.shape[1]-1]
    trainY = df_tot.iloc[:,-1]

    dim=trainX.shape[1]


    #num of classes
    num_classes=100

    trainX=np.array(trainX)
    trainY=np.array(trainY)
    
    trainY = to_categorical(trainY)


    return trainX, trainY, dim

In [236]:
def load_target_data(dataset, test_ratio):
    x, y, dim = transform_puchase_data(dataset)
    
    #trainX,testX, trainY, testY = train_test_split(x, y, test_size=test_ratio, random_state=0, stratify=y)
    trainX = x[0:14000,]
    testX = x[14000:,]
    trainY = y[0:14000,]
    testY = y[14000:,]
    return (trainX, trainY), (testX, testY), dim

In [237]:
def build_purchase_dnn(n_class,dim):
    model = Sequential()
    
    model.add(Dense(600, input_dim=dim))
    model.add(Activation("tanh"))
    #model.add(Dropout(0.01))
    
    #model.add(Dense(1024), kernel_regularizer=l2(0.001))
    #model.add(Activation("tanh"))
    #model.add(Dropout(0.01))
    
    model.add(Dense(512, kernel_regularizer=l2(0.00003)))
    model.add(Activation("tanh"))
    #model.add(Dropout(0.01))

    model.add(Dense(256, kernel_regularizer=l2(0.00003)))
    model.add(Activation("tanh"))
    #model.add(Dropout(0.01))
    
    model.add(Dense(128, kernel_regularizer=l2(0.00003)))
    model.add(Activation("tanh"))
    #model.add(Dropout(0.01))
    
    model.add(Dense(n_class, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    
    #opt = SGD(lr=0.01, momentum=0.9)
    #model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

    act_layer=6
    
    return model, act_layer

In [238]:
def build_simple_mlp(n_class,pix,d):

    model = Sequential()
    model.add(Dense(256, input_dim=pix))
    model.add(Activation("relu"))
    #model.add(Dropout(0.01))
    
    model.add(Dense(256, kernel_regularizer=l2(0.01)))
    model.add(Activation("relu"))
    #model.add(Dropout(0.01))
    
    
    #model.add(Dense(248))
    #model.add(Activation("relu"))
    #model.add(Dropout(0.01))

    model.add(Dense(64))
    model.add(Activation("relu"))
    model.add(Dropout(0.01))
    
    model.add(Dense(n_class, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    act_layer=3
    
    return model, act_layer

In [239]:
def load_shadow_data(dataset, n_shadow, shadow_size, test_ratio):
    x, y, _ = transform_puchase_data(dataset)
    
    shadow_indices = np.arange(len(dataset))
    
   
    for i in range(n_shadow):
        shadow_i_indices = np.random.choice(shadow_indices, shadow_size, replace=False)
        shadow_i_x, shadow_i_y = x[shadow_i_indices], y[shadow_i_indices]
        trainX,testX, trainY, testY = train_test_split(shadow_i_x, shadow_i_y, test_size=test_ratio)
        print('shadow_i_trainX = ', len(trainX), 'shadow_i_trainY = ', len(trainY), 'shadow_i_testX = ', len(testX), 'shadow_i_testY = ', len(testY))
        
        np.savez(DATA_PATH + 'shadow_purchase{}_data.npz'.format(i), trainX, trainY, testX, testY)

In [249]:
def train_shadow_models(n_shadow, n_class, dim, channel):
    full_sm_train_pred=[]
    full_sm_train_class=[]
    
    full_sm_test_pred=[]
    full_sm_test_class=[]
    
    full_clz_train=[]
    full_clz_test=[]
    
    members=[]
    nonmembers=[]

    for j in range(n_shadow):
        
        print("Shadow Model ", j)
        
        print('Training shadow model {}'.format(j))
        data = read_data('shadow_purchase{}_data.npz'.format(j))
        x_shadow_train, y_shadow_train, x_shadow_test, y_shadow_test = data
        #print('x_shadow trian\n', x_shadow_train,'\n y_shadow trian\n', y_shadow_train, '\n x_shadow test\n', x_shadow_test, '\n y_shadow test\n', y_shadow_test)

        model, act_layer = build_purchase_dnn(n_class,dim)
            
            
        # fit model
        history = model.fit(x_shadow_train, y_shadow_train, epochs=EPS, batch_size=32, validation_data=(x_shadow_test, y_shadow_test), verbose=0)
    
        # evaluate model
        _, train_acc = model.evaluate(x_shadow_train, y_shadow_train, verbose=0)
        _, test_acc = model.evaluate(x_shadow_test, y_shadow_test, verbose=0)
        print("Shadow Train acc : ", (train_acc * 100.0),"Shadow Test acc : ", (test_acc * 100.0))

    
        #train SM
        sm_train_pred=model.predict(x_shadow_train, batch_size=32)
        sm_train_class=np.argmax(y_shadow_train,axis=1)
    
    
        #test SM
        sm_test_pred=model.predict(x_shadow_test, batch_size=32)
        sm_test_class=np.argmax(y_shadow_test,axis=1)
        
     
        full_sm_train_pred.append(sm_train_pred)        
        full_sm_train_class.append(sm_train_class)
        members.append(np.ones(len(sm_train_pred)))
        
        full_sm_test_pred.append(sm_test_pred)        
        full_sm_test_class.append(sm_test_class) 
        nonmembers.append(np.zeros(len(sm_test_pred)))


    full_sm_train_pred = np.vstack(full_sm_train_pred)
    full_sm_train_class = [item for sublist in full_sm_train_class for item in sublist]
    members = [item for sublist in members for item in sublist]
    
    full_sm_test_pred = np.vstack(full_sm_test_pred)
    full_sm_test_class = [item for sublist in full_sm_test_class for item in sublist]
    nonmembers = [item for sublist in nonmembers for item in sublist]
    
    shadow_train_performance=(full_sm_train_pred, np.array(full_sm_train_class))
    shadow_test_performance=(full_sm_test_pred, np.array(full_sm_test_class))


    ###atack data preparation
    attack_x = (full_sm_train_pred,full_sm_test_pred)
    #attack_x = np.vstack(attack_x)
    
    attack_y = (np.array(members).astype('int32'),np.array(nonmembers).astype('int32'))
    #attack_y = np.concatenate(attack_y)
    #attack_y = attack_y.astype('int32')
    
    
    classes = (np.array(full_sm_train_class),np.array(full_sm_test_class))
    #classes = np.array([item for sublist in classes for item in sublist])


    attack_dataset = (attack_x,attack_y,classes)

            
    return  shadow_train_performance, shadow_test_performance, attack_dataset, x_shadow_train, y_shadow_train, x_shadow_test, y_shadow_test, model

In [241]:
def define_attack_model(n_class):
    model = Sequential()
    
    model.add(Dense(1))
    model.add(Activation("relu"))
    
    model.add(Dense(1))
    model.add(Activation("relu"))

    model.add(Dense(n_class, activation='softmax'))

    # compile model
    opt = SGD(learning_rate=0.0001, momentum=0.9)
    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [242]:
def load_attack_test_data(members, nonmembers):
    memberX, memberY, _ = transform_puchase_data(members)
    
    nonmemberX, nonmemberY, _ = transform_puchase_data(nonmembers)
    
    return memberX, memberY, nonmemberX, nonmemberY

In [243]:
def prety_print_result(mem, pred):
    tn, fp, fn, tp = confusion_matrix(mem, pred).ravel()
    print('TP: %d     FP: %d     FN: %d     TN: %d' % (tp, fp, fn, tn))
    if tp == fp == 0:
        print('PPV: 0\nAdvantage: 0')
    else:
        print('PPV: %.4f\nAdvantage: %.4f' % (tp / (tp + fp), tp / (tp + fn) - fp / (tn + fp)))

    return tp, fp, fn, tn, (tp / (tp + fp)), (tp / (tp + fn) - fp / (tn + fp)), ((tp+tn)/(tp+tn+fp+fn)),  (tp / (tp + fn))

In [244]:
def train_attack_model(attack_data, check_membership, n_hidden=50, learning_rate=0.01, batch_size=200, epochs=50, model='nn', l2_ratio=1e-7):

    x, y,  classes = attack_data

    train_x = x[0]
    train_y = y[0]
    test_x = x[1]
    test_y = y[1]
    train_classes = classes[0]
    test_classes = classes[1]
    
    
    checkmem_prediction_vals, checkmem_membership_status, checkmem_class_status = check_membership
    
    checkmem_prediction_vals=np.vstack(checkmem_prediction_vals)
    checkmem_membership_status=np.array([item for sublist in checkmem_membership_status for item in sublist])
    checkmem_class_status=np.array([item for sublist in checkmem_class_status for item in sublist])
    
    train_indices = np.arange(len(train_x))
    test_indices = np.arange(len(test_x))
    unique_classes = np.unique(train_classes)


    predicted_membership, target_membership = [], []
    for c in unique_classes:
        print("Class : ", c)
        c_train_indices = train_indices[train_classes == c]
        c_train_x, c_train_y = train_x[c_train_indices], train_y[c_train_indices]
        c_test_indices = test_indices[test_classes == c]
        c_test_x, c_test_y = test_x[c_test_indices], test_y[c_test_indices]
        c_dataset = (c_train_x, c_train_y, c_test_x, c_test_y)        
        
        full_cx_data=(c_train_x,c_test_x)
        full_cx_data = np.vstack(full_cx_data)

        full_cy_data=(c_train_y,c_test_y)
        full_cy_data = np.array([item for sublist in full_cy_data for item in sublist])
        full_cy_data = to_categorical(full_cy_data)

                
        
        classifier = define_attack_model(2)
        history = classifier.fit(full_cx_data, full_cy_data, epochs=EPS, batch_size=32, verbose=0)

        #get predictions on real train and test data
        c_indices = np.where(checkmem_class_status==c)
        pred_y = classifier.predict(checkmem_prediction_vals[c_indices])
        print(pred_y)
        c_pred_y = np.argmax(pred_y, axis=1)
        c_target_y = checkmem_membership_status[c_indices]
        
       
        target_membership.append(c_target_y)
        predicted_membership.append(c_pred_y)

    target_membership=np.array([item for sublist in target_membership for item in sublist])
    predicted_membership=np.array([item for sublist in predicted_membership for item in sublist])


    tp, fp, fn, tn, precision, advj, acc, recall = prety_print_result (target_membership,predicted_membership)   
    return tp, fp, fn, tn, precision, advj, acc, recall, target_membership, predicted_membership

In [245]:
# trian target model
per_class_sample=150
channel=0   
EPS=50
act_layer=6
n_class = 100
VERBOSE = 0
test_ratio = 0.3

(target_trainX, target_trainY), (target_testX, target_testY), dim = load_target_data(target_dataset, test_ratio)
target_model,_ = buid_purchase_dnn(n_class,dim)
#get_trained_keras_models(model, (target_trainX, target_trainY), (target_testX, target_testY), num_models=1)
history = target_model.fit(target_trainX, target_trainY, epochs=EPS, batch_size=32, verbose=VERBOSE)
score = target_model.evaluate(target_testX, target_testY, verbose=VERBOSE)
print('\n', 'Model test accuracy:', score[1])


 Model test accuracy: 0.7200000286102295


In [246]:
# prepare attack test data

members = []
nonmembers = []

memberX, memberY, nonmemberX, nonmemberY = load_attack_test_data(attack_test_members, attack_test_nonmembers)

# member
target_model_member_pred = target_model.predict(memberX, batch_size=32)
target_model_member_class = np.argmax(memberY, axis=1)
target_model_member_pred = np.vstack(target_model_member_pred)
#target_model_member_class = [item for sublist in target_model_member_class for item in sublist]
members.append(np.ones(len(target_model_member_pred)))
members = [item for sublist in members for item in sublist]


# nonmember
target_model_nonmember_pred = target_model.predict(nonmemberX, batch_size=32)
target_model_nonmember_class = np.argmax(nonmemberY, axis=1)
target_model_nonmember_pred = np.vstack(target_model_nonmember_pred)
#target_model_nonmember_class = [item for sublist in target_model_nonmember_class for item in sublist]
nonmembers.append(np.zeros(len(target_model_nonmember_pred)))
nonmembers = [item for sublist in nonmembers for item in sublist]

full_attack_test_pred_val = (target_model_member_pred, target_model_nonmember_pred)
full_attack_test_mem_status = (np.array(members).astype('int32'),np.array(nonmembers).astype('int32'))
full_attack_test_class_status = (np.array(target_model_member_class),np.array(target_model_nonmember_class))

print('\n pred', full_attack_test_pred_val)
print('\n class', full_attack_test_class_status)
print('\n mem status', full_attack_test_mem_status)

attack_test_data = (full_attack_test_pred_val, full_attack_test_mem_status,full_attack_test_class_status)

313/313 [==============================] - 1s 3ms/step

 pred (array([[1.3668361e-09, 3.0687744e-11, 1.0019996e-09, ..., 1.6110285e-11,
        4.8880219e-11, 2.9628533e-03],
       [5.0343079e-08, 8.0698392e-11, 4.9678738e-07, ..., 2.3774769e-15,
        1.9345644e-15, 6.3591824e-15],
       [7.4454261e-13, 4.3649684e-13, 2.8644825e-14, ..., 3.6563141e-11,
        7.3129757e-12, 9.0038416e-14],
       ...,
       [2.6256561e-09, 7.5835246e-14, 4.2797004e-07, ..., 4.8263985e-14,
        9.4105350e-16, 8.6217327e-09],
       [8.8591607e-13, 5.7765065e-14, 5.2899357e-13, ..., 7.0714284e-13,
        5.0801643e-13, 3.1914880e-15],
       [6.4447100e-05, 1.8015882e-06, 4.7187936e-03, ..., 6.8661944e-13,
        3.3541250e-09, 1.7368836e-13]], dtype=float32), array([[3.5616536e-12, 7.5151402e-06, 6.9510236e-10, ..., 2.1968416e-09,
        4.2756888e-08, 1.2168228e-11],
       [9.6744102e-10, 2.0222744e-13, 1.9613209e-08, ..., 3.7343696e-12,
        9.8180708e-13, 1.6349587e-11],
       [1.06

In [ ]:
# save attack test dataset to csv
df = pd.DataFrame()
mem = pd.Series(full_attack_test_pred_val[0][:,0])
nonmem = pd.Series(full_attack_test_pred_val[1][:,0])
total = pd.concat([mem, nonmem],axis=0, ignore_index=True)
len(total)
df['prob_upper_bound'] = total
df = df.reset_index()
mem = pd.Series(full_attack_test_pred_val[0][:,1])
nonmem = pd.Series(full_attack_test_pred_val[1][:,1])
total = pd.concat([mem, nonmem],axis=0, ignore_index=True)
df['prob_lower_bound'] = total

mem = pd.Series(full_attack_test_mem_status[0][:])
nonmem = pd.Series(full_attack_test_mem_status[1][:])
total = pd.concat([mem, nonmem],axis=0, ignore_index=True)
df['mem_status'] = total

mem = pd.Series(full_attack_test_class_status[0][:])
nonmem = pd.Series(full_attack_test_class_status[1][:])
total = pd.concat([mem, nonmem],axis=0, ignore_index=True)
df['class_status'] = total

df.drop("index", axis=1, inplace=True)


df.to_csv('attack_test_data.csv')

In [247]:
#prepare shadow dataset
n_shadow_models = 20
shadow_data_size = 10000
test_ratio = 0.3

load_shadow_data(shadow_dataset, n_shadow_models, shadow_data_size, test_ratio)

shadow_i_trainX =  7000 shadow_i_trainY =  7000 shadow_i_testX =  3000 shadow_i_testY =  3000
shadow_i_trainX =  7000 shadow_i_trainY =  7000 shadow_i_testX =  3000 shadow_i_testY =  3000
shadow_i_trainX =  7000 shadow_i_trainY =  7000 shadow_i_testX =  3000 shadow_i_testY =  3000
shadow_i_trainX =  7000 shadow_i_trainY =  7000 shadow_i_testX =  3000 shadow_i_testY =  3000
shadow_i_trainX =  7000 shadow_i_trainY =  7000 shadow_i_testX =  3000 shadow_i_testY =  3000
shadow_i_trainX =  7000 shadow_i_trainY =  7000 shadow_i_testX =  3000 shadow_i_testY =  3000
shadow_i_trainX =  7000 shadow_i_trainY =  7000 shadow_i_testX =  3000 shadow_i_testY =  3000
shadow_i_trainX =  7000 shadow_i_trainY =  7000 shadow_i_testX =  3000 shadow_i_testY =  3000
shadow_i_trainX =  7000 shadow_i_trainY =  7000 shadow_i_testX =  3000 shadow_i_testY =  3000
shadow_i_trainX =  7000 shadow_i_trainY =  7000 shadow_i_testX =  3000 shadow_i_testY =  3000
shadow_i_trainX =  7000 shadow_i_trainY =  7000 shadow_i_tes

In [250]:
#train shadow model
per_class_sample=150
channel=0   
EPS=50
act_layer=6
n_class = 100
VERBOSE = 0
test_ratio = 0.3

n_shadow_train_performance, n_shadow_test_performance, n_attack_data, x_shadow_train, y_shadow_train, x_shadow_test, y_shadow_test, shadow_model_init = train_shadow_models(n_shadow_models, n_class, dim, channel)

Shadow Model  0
Training shadow model 0
Shadow Train acc :  100.0 Shadow Test acc :  72.33333587646484
94/94 [==============================] - 0s 3ms/step
Shadow Model  1
Training shadow model 1
Shadow Train acc :  100.0 Shadow Test acc :  72.96666502952576
94/94 [==============================] - 0s 3ms/step
Shadow Model  2
Training shadow model 2
Shadow Train acc :  100.0 Shadow Test acc :  72.2000002861023
94/94 [==============================] - 0s 3ms/step
Shadow Model  3
Training shadow model 3
Shadow Train acc :  99.32857155799866 Shadow Test acc :  69.59999799728394
94/94 [==============================] - 0s 4ms/step
Shadow Model  4
Training shadow model 4
Shadow Train acc :  82.2857141494751 Shadow Test acc :  57.06666707992554
94/94 [==============================] - 0s 3ms/step
Shadow Model  5
Training shadow model 5
Shadow Train acc :  100.0 Shadow Test acc :  72.96666502952576
94/94 [==============================] - 0s 3ms/step
Shadow Model  6
Training shadow model 6
Sh

In [260]:
n_attack_data

((array([[1.18445018e-10, 1.06854777e-11, 7.14068041e-11, ...,
          3.43419598e-11, 8.86997212e-11, 4.71755524e-09],
         [2.01901074e-09, 2.84917628e-11, 8.09575340e-07, ...,
          3.47692551e-13, 6.97600581e-15, 1.64658183e-11],
         [2.61436224e-08, 1.99104441e-10, 1.43337447e-05, ...,
          3.26480943e-12, 8.75544591e-13, 8.45974013e-10],
         ...,
         [1.84684945e-09, 5.85457016e-10, 1.05127505e-07, ...,
          1.39045628e-08, 1.87799887e-09, 2.56557911e-07],
         [9.76923275e-07, 3.66467382e-13, 1.66423447e-07, ...,
          6.78800696e-11, 1.51339458e-13, 1.60510627e-08],
         [1.63107838e-09, 7.74829445e-09, 1.61906766e-09, ...,
          5.09795973e-05, 1.94620006e-04, 2.93315337e-11]], dtype=float32),
  array([[2.4440097e-10, 4.8741640e-08, 1.2910212e-10, ..., 1.3816192e-06,
          1.3464938e-06, 6.7386618e-06],
         [2.2784941e-07, 5.1086170e-11, 2.8348046e-07, ..., 5.3795823e-10,
          3.3251537e-11, 1.0062540e-06],
     

In [251]:
#save the prepared attack data on disk
np.savez(DATA_PATH + 'attack_purchase_data.npz', n_attack_data, x_shadow_train, y_shadow_train, x_shadow_test, y_shadow_test, shadow_model_init)

/home/msnkhan/anaconda3/envs/r4-base/lib/python3.7/site-packages/numpy/lib/npyio.py:719: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  val = np.asanyarray(val)


INFO:tensorflow:Assets written to: ram://584979cb-39f6-4df5-9a8a-d449090a0226/assets


In [ ]:
# read stored attack model training data 
data_name = 'attack_adult_data.npz'
with np.load(DATA_PATH + data_name, allow_pickle=True) as f:
        n_attack_data, x_shadow_train, y_shadow_train, x_shadow_test, y_shadow_test, shadow_model_init = [f['arr_%d' % i] for i in range(len(f.files))]

In [252]:
tp, fp, fn, tn, precision, advj, acc, recall, target_membership, predicted_membership = train_attack_model(n_attack_data, attack_test_data)
#target_membership, predicted_membership = train_attack_model(n_attack_data, attack_test_data)

Class :  0
5/5 [==============================] - 0s 2ms/step
[[0.35380358 0.6461964 ]
 [0.35377783 0.6462222 ]
 [0.35379416 0.64620584]
 [0.35378918 0.64621073]
 [0.353802   0.646198  ]
 [0.35379496 0.64620507]
 [0.35377985 0.64622015]
 [0.35377884 0.64622104]
 [0.35377836 0.6462217 ]
 [0.35378152 0.64621854]
 [0.35385695 0.64614314]
 [0.353782   0.646218  ]
 [0.3537832  0.6462168 ]
 [0.35378304 0.6462169 ]
 [0.35380095 0.64619905]
 [0.35378104 0.6462189 ]
 [0.3537789  0.64622104]
 [0.3537776  0.6462225 ]
 [0.35377854 0.6462214 ]
 [0.35378033 0.6462196 ]
 [0.35378975 0.6462103 ]
 [0.3537807  0.6462192 ]
 [0.3538192  0.6461808 ]
 [0.3537769  0.64622307]
 [0.3534497  0.64655024]
 [0.35377797 0.646222  ]
 [0.3537787  0.6462213 ]
 [0.35378256 0.64621735]
 [0.35377812 0.64622194]
 [0.3537808  0.6462191 ]
 [0.35718837 0.6428118 ]
 [0.3537773  0.6462226 ]
 [0.35378006 0.6462199 ]
 [0.35377994 0.64621997]
 [0.35378313 0.6462169 ]
 [0.35378182 0.6462181 ]
 [0.35378733 0.6462126 ]
 [0.3537765  

4/4 [==============================] - 0s 3ms/step
[[0.3891519  0.6108481 ]
 [0.3891519  0.6108481 ]
 [0.3891519  0.6108481 ]
 [0.3891519  0.6108481 ]
 [0.3891519  0.6108481 ]
 [0.3891519  0.6108481 ]
 [0.3891519  0.6108481 ]
 [0.3891519  0.6108481 ]
 [0.3891519  0.6108481 ]
 [0.3891519  0.6108481 ]
 [0.3891519  0.6108481 ]
 [0.3891519  0.6108481 ]
 [0.3891519  0.6108481 ]
 [0.3891519  0.6108481 ]
 [0.3891519  0.6108481 ]
 [0.3891519  0.6108481 ]
 [0.3891519  0.6108481 ]
 [0.3891519  0.6108481 ]
 [0.3891519  0.6108481 ]
 [0.3891519  0.6108481 ]
 [0.3891519  0.6108481 ]
 [0.3891519  0.6108481 ]
 [0.3891519  0.6108481 ]
 [0.3891519  0.6108481 ]
 [0.3891519  0.6108481 ]
 [0.3891519  0.6108481 ]
 [0.3891519  0.6108481 ]
 [0.3891519  0.6108481 ]
 [0.3891519  0.6108481 ]
 [0.3891519  0.6108481 ]
 [0.3891519  0.6108481 ]
 [0.3891519  0.6108481 ]
 [0.3891519  0.6108481 ]
 [0.3891519  0.6108481 ]
 [0.3891519  0.6108481 ]
 [0.3891519  0.6108481 ]
 [0.3891519  0.6108481 ]
 [0.3891519  0.6108481 ]

6/6 [==============================] - 0s 3ms/step
[[0.35565835 0.64434165]
 [0.35565835 0.64434165]
 [0.35565835 0.64434165]
 [0.35565835 0.64434165]
 [0.35565835 0.64434165]
 [0.35565835 0.64434165]
 [0.35565835 0.64434165]
 [0.35565835 0.64434165]
 [0.35565835 0.64434165]
 [0.35565835 0.64434165]
 [0.35565835 0.64434165]
 [0.35565835 0.64434165]
 [0.35565835 0.64434165]
 [0.35565835 0.64434165]
 [0.35565835 0.64434165]
 [0.35565835 0.64434165]
 [0.35565835 0.64434165]
 [0.35565835 0.64434165]
 [0.35565835 0.64434165]
 [0.35565835 0.64434165]
 [0.35565835 0.64434165]
 [0.35565835 0.64434165]
 [0.35565835 0.64434165]
 [0.35565835 0.64434165]
 [0.35565835 0.64434165]
 [0.35565835 0.64434165]
 [0.35565835 0.64434165]
 [0.35565835 0.64434165]
 [0.35565835 0.64434165]
 [0.35565835 0.64434165]
 [0.35565835 0.64434165]
 [0.35565835 0.64434165]
 [0.35565835 0.64434165]
 [0.35565835 0.64434165]
 [0.35565835 0.64434165]
 [0.35565835 0.64434165]
 [0.35565835 0.64434165]
 [0.35565835 0.64434165]

7/7 [==============================] - 0s 2ms/step
[[0.27132052 0.72867954]
 [0.27078635 0.7292136 ]
 [0.27065575 0.7293442 ]
 [0.2706727  0.7293273 ]
 [0.27065277 0.7293471 ]
 [0.27065587 0.7293442 ]
 [0.27063152 0.7293685 ]
 [0.27065283 0.7293471 ]
 [0.27079275 0.7292073 ]
 [0.27072182 0.72927815]
 [0.27077913 0.7292209 ]
 [0.27092826 0.7290717 ]
 [0.27095765 0.7290423 ]
 [0.27069932 0.72930074]
 [0.30040142 0.69959855]
 [0.27071568 0.7292844 ]
 [0.27070576 0.7292943 ]
 [0.2706366  0.7293633 ]
 [0.2715894  0.7284106 ]
 [0.27065814 0.72934186]
 [0.27308658 0.72691345]
 [0.27065036 0.7293497 ]
 [0.27112076 0.7288792 ]
 [0.27070895 0.7292911 ]
 [0.27064064 0.7293593 ]
 [0.27064943 0.7293505 ]
 [0.2707294  0.72927064]
 [0.27140513 0.7285949 ]
 [0.2707738  0.7292261 ]
 [0.27069452 0.72930545]
 [0.27076566 0.72923434]
 [0.2706973  0.72930264]
 [0.2706738  0.7293262 ]
 [0.2717482  0.7282517 ]
 [0.2707819  0.72921807]
 [0.270683   0.72931707]
 [0.27063066 0.7293693 ]
 [0.27084517 0.72915477]

6/6 [==============================] - 0s 1ms/step
[[0.35508224 0.6449178 ]
 [0.35508224 0.6449178 ]
 [0.35508224 0.6449178 ]
 [0.35508224 0.6449178 ]
 [0.35508224 0.6449178 ]
 [0.35508224 0.6449178 ]
 [0.35508224 0.6449178 ]
 [0.35508224 0.6449178 ]
 [0.35508224 0.6449178 ]
 [0.35508224 0.6449178 ]
 [0.35508224 0.6449178 ]
 [0.35508224 0.6449178 ]
 [0.35508224 0.6449178 ]
 [0.35508224 0.6449178 ]
 [0.35508224 0.6449178 ]
 [0.35508224 0.6449178 ]
 [0.35508224 0.6449178 ]
 [0.35508224 0.6449178 ]
 [0.35508224 0.6449178 ]
 [0.35508224 0.6449178 ]
 [0.35508224 0.6449178 ]
 [0.35508224 0.6449178 ]
 [0.35508224 0.6449178 ]
 [0.35508224 0.6449178 ]
 [0.35508224 0.6449178 ]
 [0.35508224 0.6449178 ]
 [0.35508224 0.6449178 ]
 [0.35508224 0.6449178 ]
 [0.35508224 0.6449178 ]
 [0.35508224 0.6449178 ]
 [0.35508224 0.6449178 ]
 [0.35508224 0.6449178 ]
 [0.35508224 0.6449178 ]
 [0.35508224 0.6449178 ]
 [0.35508224 0.6449178 ]
 [0.35508224 0.6449178 ]
 [0.35508224 0.6449178 ]
 [0.35508224 0.6449178 ]

...

6/6 [==============================] - 0s 1ms/step
[[0.350828   0.64917195]
 [0.350828   0.64917195]
 [0.350828   0.64917195]
 [0.350828   0.64917195]
 [0.350828   0.64917195]
 [0.350828   0.64917195]
 [0.350828   0.64917195]
 [0.350828   0.64917195]
 [0.350828   0.64917195]
 [0.350828   0.64917195]
 [0.350828   0.64917195]
 [0.350828   0.64917195]
 [0.350828   0.64917195]
 [0.350828   0.64917195]
 [0.350828   0.64917195]
 [0.350828   0.64917195]
 [0.350828   0.64917195]
 [0.350828   0.64917195]
 [0.350828   0.64917195]
 [0.350828   0.64917195]
 [0.350828   0.64917195]
 [0.350828   0.64917195]
 [0.350828   0.64917195]
 [0.350828   0.64917195]
 [0.350828   0.64917195]
 [0.350828   0.64917195]
 [0.350828   0.64917195]
 [0.350828   0.64917195]
 [0.350828   0.64917195]
 [0.350828   0.64917195]
 [0.350828   0.64917195]
 [0.350828   0.64917195]
 [0.350828   0.64917195]
 [0.350828   0.64917195]
 [0.350828   0.64917195]
 [0.350828   0.64917195]
 [0.350828   0.64917195]
 [0.350828   0.64917195]

6/6 [==============================] - 0s 1ms/step
[[0.35908464 0.64091533]
 [0.35908464 0.64091533]
 [0.35908464 0.64091533]
 [0.35908464 0.64091533]
 [0.35908464 0.64091533]
 [0.35908464 0.64091533]
 [0.35908464 0.64091533]
 [0.35908464 0.64091533]
 [0.35908464 0.64091533]
 [0.35908464 0.64091533]
 [0.35908464 0.64091533]
 [0.35908464 0.64091533]
 [0.35908464 0.64091533]
 [0.35908544 0.6409145 ]
 [0.35908464 0.64091533]
 [0.35908464 0.64091533]
 [0.35908464 0.64091533]
 [0.35908464 0.64091533]
 [0.35908464 0.64091533]
 [0.35908464 0.64091533]
 [0.35908464 0.64091533]
 [0.35908854 0.64091146]
 [0.35908464 0.64091533]
 [0.35908464 0.64091533]
 [0.35908464 0.64091533]
 [0.35908464 0.64091533]
 [0.35908464 0.64091533]
 [0.35909337 0.64090663]
 [0.35930008 0.64069986]
 [0.35908464 0.64091533]
 [0.35908464 0.64091533]
 [0.35909092 0.640909  ]
 [0.35908464 0.64091533]
 [0.35908464 0.64091533]
 [0.35908464 0.64091533]
 [0.35908464 0.64091533]
 [0.35908464 0.64091533]
 [0.35908464 0.64091533]

6/6 [==============================] - 0s 2ms/step
[[0.3388933  0.6611068 ]
 [0.3388933  0.6611068 ]
 [0.3388933  0.6611068 ]
 [0.3388933  0.6611068 ]
 [0.3388933  0.6611068 ]
 [0.3388933  0.6611068 ]
 [0.3388933  0.6611068 ]
 [0.3388933  0.6611068 ]
 [0.3388933  0.6611068 ]
 [0.3388933  0.6611068 ]
 [0.3388933  0.6611068 ]
 [0.3388933  0.6611068 ]
 [0.3388933  0.6611068 ]
 [0.3388933  0.6611068 ]
 [0.3388933  0.6611068 ]
 [0.3388933  0.6611068 ]
 [0.3388933  0.6611068 ]
 [0.3388933  0.6611068 ]
 [0.3388933  0.6611068 ]
 [0.3388933  0.6611068 ]
 [0.3388933  0.6611068 ]
 [0.3388933  0.6611068 ]
 [0.3388933  0.6611068 ]
 [0.3388933  0.6611068 ]
 [0.3388933  0.6611068 ]
 [0.3388933  0.6611068 ]
 [0.3388933  0.6611068 ]
 [0.3388933  0.6611068 ]
 [0.3388933  0.6611068 ]
 [0.3388933  0.6611068 ]
 [0.3388933  0.6611068 ]
 [0.3388933  0.6611068 ]
 [0.3388933  0.6611068 ]
 [0.3388933  0.6611068 ]
 [0.3388933  0.6611068 ]
 [0.3388933  0.6611068 ]
 [0.3388933  0.6611068 ]
 [0.3388933  0.6611068 ]

3/3 [==============================] - 0s 2ms/step
[[0.42678198 0.5732179 ]
 [0.42678198 0.5732179 ]
 [0.42678198 0.5732179 ]
 [0.42678198 0.5732179 ]
 [0.42678198 0.5732179 ]
 [0.42678198 0.5732179 ]
 [0.42678198 0.5732179 ]
 [0.42678198 0.5732179 ]
 [0.42678198 0.5732179 ]
 [0.42678198 0.5732179 ]
 [0.42678198 0.5732179 ]
 [0.42678198 0.5732179 ]
 [0.42678198 0.5732179 ]
 [0.42678198 0.5732179 ]
 [0.42678198 0.5732179 ]
 [0.42678198 0.5732179 ]
 [0.42678198 0.5732179 ]
 [0.42678198 0.5732179 ]
 [0.42678198 0.5732179 ]
 [0.42678198 0.5732179 ]
 [0.42678198 0.5732179 ]
 [0.42678198 0.5732179 ]
 [0.42678198 0.5732179 ]
 [0.42678198 0.5732179 ]
 [0.42678198 0.5732179 ]
 [0.42678198 0.5732179 ]
 [0.42678198 0.5732179 ]
 [0.42678198 0.5732179 ]
 [0.42678198 0.5732179 ]
 [0.42678198 0.5732179 ]
 [0.42678198 0.5732179 ]
 [0.42678198 0.5732179 ]
 [0.42678198 0.5732179 ]
 [0.42678198 0.5732179 ]
 [0.42678198 0.5732179 ]
 [0.42678198 0.5732179 ]
 [0.42678198 0.5732179 ]
 [0.42678198 0.5732179 ]

7/7 [==============================] - 0s 2ms/step
[[0.3450469  0.65495306]
 [0.3450469  0.65495306]
 [0.3450469  0.65495306]
 [0.3450469  0.65495306]
 [0.3450469  0.65495306]
 [0.3450469  0.65495306]
 [0.3450469  0.65495306]
 [0.3450469  0.65495306]
 [0.3450469  0.65495306]
 [0.3450469  0.65495306]
 [0.3450469  0.65495306]
 [0.3450469  0.65495306]
 [0.3450469  0.65495306]
 [0.3450469  0.65495306]
 [0.3450469  0.65495306]
 [0.3450469  0.65495306]
 [0.3450469  0.65495306]
 [0.3450469  0.65495306]
 [0.3450469  0.65495306]
 [0.3450469  0.65495306]
 [0.3450469  0.65495306]
 [0.3450469  0.65495306]
 [0.3450469  0.65495306]
 [0.3450469  0.65495306]
 [0.3450469  0.65495306]
 [0.3450469  0.65495306]
 [0.3450469  0.65495306]
 [0.3450469  0.65495306]
 [0.3450469  0.65495306]
 [0.3450469  0.65495306]
 [0.3450469  0.65495306]
 [0.3450469  0.65495306]
 [0.3450469  0.65495306]
 [0.3450469  0.65495306]
 [0.3450469  0.65495306]
 [0.3450469  0.65495306]
 [0.3450469  0.65495306]
 [0.3450469  0.65495306]

In [259]:
precision

0.5004757373929591

In [ ]:
target_membership

In [ ]:
predicted_membership

In [ ]:
df = pd.DataFrame()
df['target'] = pd.Series(target_membership)
df['predicted'] = pd.Series(predicted_membership)

In [ ]:
df

In [ ]:
df.to_csv('test.csv')

In [ ]:
attack_data = n_attack_data
check_membership = attack_test_data
n_hidden=50
learning_rate=0.01
batch_size=200
epochs=50
model='nn'
l2_ratio=1e-7

In [ ]:
x, y,  classes = attack_data

train_x = x[0]
train_y = y[0]
test_x = x[1]
test_y = y[1]
train_classes = classes[0]
test_classes = classes[1]

#print(tra)


In [ ]:
checkmem_prediction_vals, checkmem_membership_status, checkmem_class_status = check_membership

checkmem_prediction_vals=np.vstack(checkmem_prediction_vals)
checkmem_membership_status=np.array([item for sublist in checkmem_membership_status for item in sublist])
checkmem_class_status=np.array([item for sublist in checkmem_class_status for item in sublist])

train_indices = np.arange(len(train_x))
test_indices = np.arange(len(test_x))
unique_classes = np.unique(train_classes)


predicted_membership, target_membership = [], []

In [ ]:
for c in unique_classes:
    print("Class : ", c)
    c_train_indices = train_indices[train_classes == c]
    c_train_x, c_train_y = train_x[c_train_indices], train_y[c_train_indices]
    c_test_indices = test_indices[test_classes == c]
    c_test_x, c_test_y = test_x[c_test_indices], test_y[c_test_indices]
    c_dataset = (c_train_x, c_train_y, c_test_x, c_test_y)  

    full_cx_data=(c_train_x,c_test_x)
    full_cx_data = np.vstack(full_cx_data)

    full_cy_data=(c_train_y,c_test_y)
    full_cy_data = np.array([item for sublist in full_cy_data for item in sublist])
    full_cy_data = to_categorical(full_cy_data)

    classifier = define_attack_model(2)
    history = classifier.fit(full_cx_data, full_cy_data, epochs=200, batch_size=32, verbose=0)
    #classifier.save('model/attack_model_class{}'.format(c))

    #get predictions on real train and test data
    c_indices = np.where(checkmem_class_status==c)
    predict_y = classifier.predict(checkmem_prediction_vals[c_indices])
    print(predict_y)
    c_pred_y = np.argmax(classifier.predict(checkmem_prediction_vals[c_indices]),axis=1)
    #c_pred_y = np.where(predict_y > 0.5, 1,0)
    #c_pred_y = classifier.predict_classes(checkmem_prediction_vals[c_indices])

    c_target_y = checkmem_membership_status[c_indices]


    target_membership.append(c_target_y)
    predicted_membership.append(c_pred_y)




In [ ]:
c_indices = np.where(checkmem_class_status==c)
predict_y = classifier.predict(checkmem_prediction_vals[c_indices], batch_size=32)
c_pred_y = np.where(predict_y > 0.5, 1,0)
c_target_y = checkmem_membership_status[c_indices]
target_membership.append(c_target_y)
predicted_membership.append(c_pred_y)

In [ ]:
target_membership=np.array([item for sublist in target_membership for item in sublist])
predicted_membership=np.array([item for sublist in predicted_membership for item in sublist])

In [ ]:
prety_print_result (target_membership,predicted_membership)

In [ ]:
np.vstack(predicted_membership).shape

In [ ]:
df = pd.DataFrame()
df['target'] = pd.Series(target_membership)
df['predicted'] = pd.Series(predicted_membership.reshape((len(predicted_membership))))

In [ ]:
df

In [ ]:
predicted_membership